<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) | [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) | [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/01.07-Timing-and-Profiling.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# プロファイリングとタイミング コード

コードを開発し、データ処理パイプラインを作成するプロセスでは、多くの場合、さまざまな実装間で行うことができるトレードオフがあります。
アルゴリズムの開発の早い段階で、そのようなことを心配することは逆効果になる可能性があります。 Donald Knuth が有名な皮肉を言ったように、「97% の確率で、小さな効率性は忘れるべきです。時期尚早の最適化はすべての悪の根源です。」

しかし、コードが機能するようになったら、その効率を少し掘り下げてみると便利です。
特定のコマンドまたはコマンド セットの実行時間を確認すると便利な場合があります。また、複数行のプロセスを掘り下げて、複雑な一連の操作のどこにボトルネックがあるかを判断すると便利な場合もあります。
IPython は、この種のコードのタイミングとプロファイリングのための幅広い機能へのアクセスを提供します。
ここでは、次の IPython マジック コマンドについて説明します。

- ``%time``: 単一のステートメントの実行時間
- ``%timeit``: より正確な単一ステートメントの繰り返し実行時間
- ``%prun``: プロファイラでコードを実行します
- ``%lprun``: 行ごとのプロファイラでコードを実行します
- ``%memit``: 単一のステートメントのメモリ使用量を測定します
- ``%mprun``: 行ごとのメモリプロファイラでコードを実行します

最後の 4 つのコマンドは IPython にバンドルされていません。次のセクションで説明する ``line_profiler`` および ``memory_profiler`` 拡張機能を取得する必要があります。

## タイミングコードの断片: ``%timeit`` と ``%time``

[IPython Magic Commands](01.03-Magic-Commands.ipynb) のマジック関数の紹介で、``%timeit`` ライン マジックと ``%%timeit`` セル マジックを見ました。コードのスニペットを繰り返し実行する時間を計測するために使用できます。

In [1]:
%timeit sum(range(100))

100000 loops, best of 3: 1.54 µs per loop


この操作は非常に高速であるため、 %timeit は自動的に多数の繰り返しを行うことに注意してください。
より遅いコマンドの場合、 %timeit は自動的に調整し、繰り返しを少なくします:

In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

1 loops, best of 3: 407 ms per loop


操作を繰り返すことが最善の選択肢ではない場合があります。
たとえば、並べ替えたいリストがある場合、繰り返し操作によって誤解される可能性があります。
事前に並べ替えられたリストの並べ替えは、並べ替えられていないリストの並べ替えよりもはるかに高速であるため、繰り返しによって結果が歪められます。

In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

100 loops, best of 3: 1.9 ms per loop


このためには、``%time`` マジック関数がより良い選択かもしれません。また、システム関連の短い遅延が結果に影響を与える可能性が低い場合は、実行時間の長いコマンドにも適しています。
ソートされていないリストと事前にソートされたリストのソートの時間を計りましょう:

In [4]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 40.6 ms, sys: 896 µs, total: 41.5 ms
Wall time: 41.5 ms


In [5]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 8.18 ms, sys: 10 µs, total: 8.19 ms
Wall time: 8.24 ms


事前に並べ替えられたリストのソートがどれだけ速いかに注目してください。ただし、事前に並べ替えられたリストであっても、 %time と %timeit のタイミングがどれだけ長くかかるかに注意してください!
これは、システムコールがタイミングに干渉するのを防ぐために ``%timeit`` が内部で巧妙なことを行っているという事実の結果です。
たとえば、タイミングに影響を与える可能性のある未使用の Python オブジェクト (*ガベージ コレクション* と呼ばれる) のクリーンアップを防ぎます。
このため、%timeit の結果は通常、%time の結果よりも著しく高速です。

``%timeit`` と同様に ``%time`` の場合、二重パーセント記号のセル マジック構文を使用すると、複数行のスクリプトのタイミングが可能になります。

In [6]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 504 ms, sys: 979 µs, total: 505 ms
Wall time: 505 ms


``%time`` と ``%timeit`` およびそれらの利用可能なオプションの詳細については、IPython ヘルプ機能を使用してください (つまり、IPython プロンプトで ``%time?`` と入力してください)。

## 完全なスクリプトのプロファイリング: ``%prun``

プログラムは多くの単一のステートメントで構成されており、コンテキスト内でこれらのステートメントのタイミングを計ることが、単独でタイミングを計ることよりも重要な場合があります。
Python には組み込みのコード プロファイラー (Python のドキュメントで読むことができます) が含まれていますが、IPython は魔法の関数 ``%prun`` の形で、このプロファイラーを使用するはるかに便利な方法を提供します。

例として、いくつかの計算を行う単純な関数を定義します。

In [7]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

これで、関数呼び出しで ``%prun`` を呼び出して、プロファイリングされた結果を確認できます。

In [8]:
%prun sum_of_lists(1000000)

ノートブックでは、出力はページャーに出力され、次のようになります。

```
14 function calls in 0.714 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.599    0.120    0.599    0.120 <ipython-input-19>:4(<listcomp>)
        5    0.064    0.013    0.064    0.013 {built-in method sum}
        1    0.036    0.036    0.699    0.699 <ipython-input-19>:1(sum_of_lists)
        1    0.014    0.014    0.714    0.714 <string>:1(<module>)
        1    0.000    0.000    0.714    0.714 {built-in method exec}
```

結果は、各関数呼び出しの合計時間の順に、実行に最も時間がかかっている場所を示すテーブルです。この場合、実行時間の大部分は ``sum_of_lists`` 内のリスト内包表記にあります。
ここから、アルゴリズムのパフォーマンスを向上させるためにどのような変更を加えるかについて考え始めることができます。

``%prun`` とその利用可能なオプションの詳細については、IPython ヘルプ機能を使用してください (つまり、IPython プロンプトで ``%prun?`` と入力してください)。

## ``%lprun`` による行ごとのプロファイリング

``%prun`` の関数ごとのプロファイリングは便利ですが、行ごとのプロファイル レポートがある方が便利な場合もあります。
これは Python や IPython には組み込まれていませんが、これを実行できるインストール用の ``line_profiler`` パッケージがあります。
まず、Python のパッケージング ツール pip を使用して、line_profiler パッケージをインストールします。

```
$ pip install line_profiler
```

次に、IPython を使用して、このパッケージの一部として提供される ``line_profiler`` IPython 拡張機能をロードできます。

In [9]:
%load_ext line_profiler

%lprun コマンドは、任意の関数の行ごとのプロファイリングを実行します。この場合、プロファイリングに関心のある関数を明示的に伝える必要があります。

In [10]:
%lprun -f sum_of_lists sum_of_lists(5000)

以前と同様に、ノートブックは結果をページャーに送信しますが、次のようになります。

```
Timer unit: 1e-06 s

Total time: 0.009382 s
File: <ipython-input-19-fa2be176cc3e>
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def sum_of_lists(N):
     2         1            2      2.0      0.0      total = 0
     3         6            8      1.3      0.1      for i in range(5):
     4         5         9001   1800.2     95.9          L = [j ^ (j >> i) for j in range(N)]
     5         5          371     74.2      4.0          total += sum(L)
     6         1            0      0.0      0.0      return total
```

上部の情報は、結果を読み取るための鍵となります。時間はマイクロ秒単位で報告され、プログラムが最も多くの時間を費やしている場所を確認できます。
この時点で、この情報を使用してスクリプトの側面を変更し、目的のユース ケースのパフォーマンスを向上させることができる場合があります。

``%lprun`` とその利用可能なオプションの詳細については、IPython ヘルプ機能を使用してください (つまり、IPython プロンプトで ``%lprun?`` と入力してください)。

## メモリー使用量のプロファイリング: ``%memit`` および ``%mprun``

プロファイリングのもう 1 つの側面は、操作が使用するメモリの量です。
これは、別の IPython 拡張である ``memory_profiler`` で評価できます。
``line_profiler`` と同様に、拡張機能を ``pip`` でインストールすることから始めます。

```
$ pip install memory_profiler
```

次に、IPython を使用して拡張機能をロードできます。

In [12]:
%load_ext memory_profiler

メモリープロファイラー拡張機能には、2 つの便利な魔法の関数が含まれています: ``%memit`` マジック (``%timeit`` と同等のメモリー測定を提供します) と ``%mprun`` 関数 (メモリー測定を提供します)。 ``%lprun`` と同等)。
``%memit`` 関数はかなり単純に使用できます:

In [13]:
%memit sum_of_lists(1000000)

peak memory: 100.08 MiB, increment: 61.36 MiB


この関数は約 100 MB のメモリを使用することがわかります。

メモリ使用量の行ごとの説明については、``%mprun`` マジックを使用できます。
残念ながら、この魔法はノートブック自体ではなく、別のモジュールで定義された関数に対してのみ機能します。 sum_of_lists 関数に、メモリ プロファイリングの結果をより明確にする 1 つの追加を加えま​​す。

In [14]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Overwriting mprun_demo.py


この関数の新しいバージョンをインポートして、メモリ ライン プロファイラーを実行できるようになりました。

In [15]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

ページャーに出力された結果は、関数のメモリ使用の概要を示し、次のようになります。
```
Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     4     71.9 MiB      0.0 MiB           L = [j ^ (j >> i) for j in range(N)]


Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1     39.0 MiB      0.0 MiB   def sum_of_lists(N):
     2     39.0 MiB      0.0 MiB       total = 0
     3     46.5 MiB      7.5 MiB       for i in range(5):
     4     71.9 MiB     25.4 MiB           L = [j ^ (j >> i) for j in range(N)]
     5     71.9 MiB      0.0 MiB           total += sum(L)
     6     46.5 MiB    -25.4 MiB           del L # remove reference to L
     7     39.1 MiB     -7.4 MiB       return total
```
ここで ``Increment`` 列は、各行が総メモリ バジェットにどの程度影響するかを示しています: リスト ``L`` を作成および削除すると、約 25 MB のメモリ使用量が追加されることに注意してください。
これは、Python インタープリター自体からのバックグラウンド メモリ使用量に加えて発生します。

``%memit`` と ``%mprun`` およびそれらの利用可能なオプションの詳細については、IPython ヘルプ機能を使用してください (つまり、IPython プロンプトで ``%memit?`` と入力してください)。

<!--ナビゲーション-->
< [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) | [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) | [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/01.07-Timing-and-Profiling.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
